In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
!pip install nemo_toolkit['all']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install hydra-core==1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
import omegaconf
from omegaconf import OmegaConf
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.plugins import DDPPlugin
import sys
import logging

[NeMo W 2022-06-09 11:35:53 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [6]:
sys.argv = ['']
del sys

LANGUAGE = "slices"
path = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive'
manifest_dir = os.path.join(path, LANGUAGE)
train_manifest = f"{manifest_dir}/train/train.json"
dev_manifest = f"{manifest_dir}/dev/dev.json"
test_manifest = f"{manifest_dir}/test/test.json"


@hydra_runner(config_path=r"/content/drive/MyDrive/Colab Notebooks/Sprinklr_Project1/conformer", config_name="conformer_ctc_bpe")
def main(cfg):
    logging.debug(cfg)
    cfg['model']['train_ds']['manifest_filepath'] = train_manifest
    cfg['model']['validation_ds']['manifest_filepath'] = dev_manifest
    cfg['model']['test_ds']['manifest_filepath'] = test_manifest

    logging.info(f'Hydra config: {OmegaConf.to_yaml(cfg)}')
    logging.info("trainer: {}".format(cfg.trainer))
    checkpoint_callback = ModelCheckpoint(dirpath='model_output_dir', save_last=True, save_top_k=20,
                                          monitor="val_wer", every_n_epochs=10)
    checkpoint_path = None
    trainer = pl.Trainer(gpus=1, max_epochs=100, callbacks=[checkpoint_callback], auto_select_gpus=True)

    asr_model = EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")
    asr_model._wer.log_prediction = True

    asr_model.set_trainer(trainer)
    param_config = DictConfig(cfg['model'])
    asr_model.setup_training_data(param_config.train_ds)
    asr_model.setup_multiple_validation_data(val_data_config=param_config.validation_ds)
    asr_model.setup_multiple_test_data(test_data_config=param_config.test_ds)
    asr_model.spec_augmentation = asr_model.from_config_dict(asr_model.cfg.spec_augment)
    asr_model.setup_optimization(DictConfig(cfg['model']['optim']))
    asr_model.encoder.unfreeze()
    asr_model.decoder.unfreeze()

    trainer.fit(asr_model, ckpt_path=checkpoint_path)

    if hasattr(cfg.model, 'test_ds') and cfg.model.test_ds.manifest_filepath is not None and False:
        gpu = 1 if cfg.trainer.gpus != 0 else 0
        test_trainer = pl.Trainer(
            gpus=gpu,
            precision=trainer.precision,
            amp_level=trainer.accelerator_connector.amp_level,
            amp_backend=cfg.trainer.get("amp_backend", "native"),
        )
        if asr_model.prepare_test(test_trainer):
            test_trainer.test(asr_model)


if __name__ == '__main__':
    main()


Hydra config: name: Conformer-CTC-BPE
model:
  sample_rate: 16000
  log_prediction: true
  ctc_reduction: mean_batch
  train_ds:
    manifest_filepath: /content/drive/Othercomputers/My MacBook Pro/Google_Drive/slices/train/train.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
  validation_ds:
    manifest_filepath: /content/drive/Othercomputers/My MacBook Pro/Google_Drive/slices/dev/dev.json
    sample_rate: 16000
    batch_size: 16
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
  test_ds:
    manifest_filepath: /content/drive/Othercomputers/My MacBook Pro/Google_Drive/slices/test/test.json
    sample_rate: 16000
    batch_size: 16
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
  tokenizer:
    dir: ???
    type: wpe
  preprocessor:
    _target_: 

Auto select gpus: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2022-06-09 11:36:00 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.
[NeMo I 2022-06-09 11:36:00 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-06-09 11:36:00 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-09 11:36:07 mixins:168] Tokenizer SentencePieceTokenizer initialized with 128 tokens
Created a temporary directory at /tmp/tmpzb9ipasv
Writing /tmp/tmpzb9ipasv/_remote_module_non_sriptable.py


[NeMo W 2022-06-09 11:36:07 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-06-09 11:36:07 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-06-09 11:36:07 features:200] PADDING: 0


...

[NeMo I 2022-06-09 13:41:28 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:28 wer_bpe:213] reference:zero
[NeMo I 2022-06-09 13:41:28 wer_bpe:214] predicted:zero
[NeMo I 2022-06-09 13:41:28 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:28 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:41:28 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:41:29 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:29 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:41:29 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:41:29 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:29 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:41:29 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:41:29 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:29 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:41:29 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:41:29 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:41:29 wer_bpe:213] reference:five
[NeMo I 2022-06-09 13:41:29 wer_bpe:214] predicted:five
[NeMo I 2022-06-09 13:41:29 wer_bpe:212] 
    
[NeMo I

Validation: 0it [00:00, ?it/s]

[NeMo I 2022-06-09 13:42:45 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:45 wer_bpe:213] reference:zero
[NeMo I 2022-06-09 13:42:45 wer_bpe:214] predicted:zero
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:46 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:42:46 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:46 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:42:46 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:46 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:42:46 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:46 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:42:46 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:42:46 wer_bpe:213] reference:five
[NeMo I 2022-06-09 13:42:46 wer_bpe:214] predicted:five
[NeMo I 2022-06-09 13:42:46 wer_bpe:212] 
    
[NeMo I

Validation: 0it [00:00, ?it/s]

[NeMo I 2022-06-09 13:44:02 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:03 wer_bpe:213] reference:zero
[NeMo I 2022-06-09 13:44:03 wer_bpe:214] predicted:zero
[NeMo I 2022-06-09 13:44:03 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:03 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:44:03 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:44:03 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:03 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:44:03 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:44:03 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:03 wer_bpe:213] reference:two
[NeMo I 2022-06-09 13:44:03 wer_bpe:214] predicted:two
[NeMo I 2022-06-09 13:44:03 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:03 wer_bpe:213] reference:one
[NeMo I 2022-06-09 13:44:03 wer_bpe:214] predicted:one
[NeMo I 2022-06-09 13:44:04 wer_bpe:212] 
    
[NeMo I 2022-06-09 13:44:04 wer_bpe:213] reference:five
[NeMo I 2022-06-09 13:44:04 wer_bpe:214] predicted:five
[NeMo I 2022-06-09 13:44:04 wer_bpe:212] 
    
[NeMo I